In [1]:
import numpy as np
import pandas as pd

import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [10]:
with open("C:\\Users\\Nemesis Crociata\\.secret\\yelp_api.json") as f:
    login = json.load(f)

yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [17]:
LOCATION = "New Orleans, LA"
TERM = "Beignets"

In [18]:
JSON_FILE = "Data/results_in_progress_LA_beignets.json"

In [19]:
def create_json_file(JSON_FILE,  delete_if_exists=False):

    file_exists = os.path.isfile(JSON_FILE)

    if file_exists == True:

        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            os.remove(JSON_FILE)
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
    else:

        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")

        folder = os.path.dirname(JSON_FILE)

        if len(folder)>0:
            os.makedirs(folder,exist_ok=True)
        with open(JSON_FILE,'w') as f:
            json.dump([],f)

In [20]:
create_json_file(JSON_FILE, delete_if_exists=True)
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)

n_results = len(previous_results)
print(f'- {n_results} previous results found.')
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)

total_results = results['total']
results_per_page = len(results['businesses'])
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[i] Data/results_in_progress_LA_beignets.json not found. Saving empty list to new file.
- 0 previous results found.


27

In [21]:
for i in tqdm_notebook( range(1,n_pages+1)):

    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break

    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    previous_results.extend(results['businesses'])

    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/27 [00:00<?, ?it/s]

In [22]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,Y2Pfil51rNvTd_lFHwzb_g,cafe-beignet-on-royal-street-new-orleans-2,"Cafe Beignet, Royal Street",https://s3-media4.fl.yelpcdn.com/bphoto/W96CW7...,False,https://www.yelp.com/biz/cafe-beignet-on-royal...,3078,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 29.955339, 'longitude': -90.066824}",[delivery],$$,"{'address1': '334 Royal St', 'address2': '', '...",+15045245530,(504) 524-5530,1634.760242
1,FEXhWNCMkv22qG04E83Qjg,cafe-du-monde-new-orleans-21,Cafe Du Monde,https://s3-media3.fl.yelpcdn.com/bphoto/O7ZHbX...,False,https://www.yelp.com/biz/cafe-du-monde-new-orl...,3061,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"{'latitude': 29.957525, 'longitude': -90.061861}",[delivery],$,"{'address1': '800 Decatur St', 'address2': '',...",+15045254544,(504) 525-4544,2116.842756
2,MyFC2F-5SXwXTLR99UWiqA,the-vintage-new-orleans-2,The Vintage,https://s3-media2.fl.yelpcdn.com/bphoto/Ff39Fx...,False,https://www.yelp.com/biz/the-vintage-new-orlea...,488,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",4.5,"{'latitude': 29.92444, 'longitude': -90.08613}",[delivery],$$,"{'address1': '3121 Magazine St', 'address2': '...",+15043247144,(504) 324-7144,3534.907905
3,sG-Z7aIlkmKW7EstH1dH5Q,streetcar-cafe-new-orleans-2,Streetcar Cafe,https://s3-media2.fl.yelpcdn.com/bphoto/Ei3e5j...,False,https://www.yelp.com/biz/streetcar-cafe-new-or...,590,"[{'alias': 'breakfast_brunch', 'title': 'Break...",4.0,"{'latitude': 29.95111, 'longitude': -90.06992}",[delivery],$$,"{'address1': '312 St Charles Ave', 'address2':...",+15042187256,(504) 218-7256,1434.978912
4,CcAtO2dgxzG58x6sze8dlg,lorettas-authentic-pralines-new-orleans,Loretta's Authentic Pralines,https://s3-media3.fl.yelpcdn.com/bphoto/jPSJYg...,False,https://www.yelp.com/biz/lorettas-authentic-pr...,399,"[{'alias': 'candy', 'title': 'Candy Stores'}, ...",4.5,"{'latitude': 29.960141, 'longitude': -90.05898}",[],$,"{'address1': '1100 N Peters St', 'address2': '...",+15043238350,(504) 323-8350,2445.132320


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
525,86cSxahOGt6_PTL_m3S-2Q,cathay-inn-imperial-restaurant-metairie,Cathay Inn Imperial Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/lZd78p...,False,https://www.yelp.com/biz/cathay-inn-imperial-r...,50,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",3.5,"{'latitude': 29.97803159915492, 'longitude': -...",[delivery],$,"{'address1': '6501 Airline Dr', 'address2': ''...",+15047338415,(504) 733-8415,11925.721691
526,E48H5Zpw3ndlu7KLHGRyHg,ruths-chris-steak-house-metairie-2,Ruth's Chris Steak House,https://s3-media1.fl.yelpcdn.com/bphoto/lNnb-T...,False,https://www.yelp.com/biz/ruths-chris-steak-hou...,203,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.5,"{'latitude': 30.0043740015754, 'longitude': -9...","[pickup, delivery]",$$$$,"{'address1': '3633 Veterans Blvd', 'address2':...",+15048883600,(504) 888-3600,9352.935520
527,h4ghREOFsZjxqc8hUEHRBQ,fiery-crab-seafood-restaurant-and-bar-kenner-2,Fiery Crab Seafood Restaurant And Bar,https://s3-media3.fl.yelpcdn.com/bphoto/7Z3sFI...,False,https://www.yelp.com/biz/fiery-crab-seafood-re...,177,"[{'alias': 'seafood', 'title': 'Seafood'}, {'a...",2.5,"{'latitude': 30.02734667, 'longitude': -90.23832}","[pickup, delivery]",$$,"{'address1': '3900 Williams Blvd', 'address2':...",+15043323900,(504) 332-3900,16846.429285
528,zrUAjvPlH22dnvNtxRuiBg,the-cheesecake-factory-metairie-metairie-2,The Cheesecake Factory - Metairie,https://s3-media1.fl.yelpcdn.com/bphoto/SJkRPY...,False,https://www.yelp.com/biz/the-cheesecake-factor...,564,"[{'alias': 'desserts', 'title': 'Desserts'}, {...",2.5,"{'latitude': 30.0058530931022, 'longitude': -9...",[delivery],$$,"{'address1': '3301 Veterans Memorial Blvd', 'a...",+15048371818,(504) 837-1818,8904.382976
529,xCdqyXUbY0Tzfmbx9tx0YQ,panera-bread-metairie,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/38n0vl...,False,https://www.yelp.com/biz/panera-bread-metairie...,160,"[{'alias': 'soup', 'title': 'Soup'}, {'alias':...",3.0,"{'latitude': 30.004868, 'longitude': -90.190717}","[pickup, delivery]",$$,"{'address1': '4848 Veterans Boulevard', 'addre...",+15042934760,(504) 293-4760,11629.915954
